# Series Analysis Workflows

This notebook demonstrates q-Kangaroo's tools for analyzing q-series:
recovering infinite product representations, extracting arithmetic
subsequences, and discovering algebraic relations. These are the core
research workflows for studying q-series identities.

## Starting Point: A Mystery Series

Suppose we have a q-series and want to understand its structure. We use
the partition generating function as our first example -- it is well-known,
so we can verify each step.

In [1]:
from q_kangaroo import QSession, partition_gf

s = QSession()
pgf = partition_gf(s, 50)

# Display the first 25 terms
partition_gf(s, 25)

1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + 42*q^10 + 56*q^11 + 77*q^12 + 101*q^13 + 135*q^14 + 176*q^15 + 231*q^16 + 297*q^17 + 385*q^18 + 490*q^19 + 627*q^20 + 792*q^21 + 1002*q^22 + 1255*q^23 + 1575*q^24 + O(q^25)

## Prodmake: Recovering Infinite Product Structure

Andrews' `prodmake` algorithm determines exponents $a_n$ such that

$$f(q) = \prod_{n=1}^{\infty} (1-q^n)^{-a_n}$$

It works by taking the logarithmic derivative of the product,
recovering the exponents via a recurrence, then applying Mobius
inversion.

In [2]:
from q_kangaroo import prodmake

pm = prodmake(pgf, 20)
print("Prodmake factors (a_n where f = prod (1-q^n)^{-a_n}):")
print(pm["factors"])

Prodmake factors (a_n where f = prod (1-q^n)^{-a_n}):
{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1}


Every exponent $a_n = 1$, confirming the Euler product:

$$\sum p(n) q^n = \prod_{k=1}^{\infty} (1-q^k)^{-1}$$

The `prodmake` algorithm recovers this structure automatically from
the first 50 series coefficients.

## Etamake: Eta-Quotient Representation

The `etamake` function groups `prodmake` output by divisor, recovering
the Dedekind eta-quotient $\prod_d \eta(d\tau)^{r_d}$. Since
$\eta(\tau) = q^{1/24} \prod_{n=1}^{\infty}(1-q^n)$, the eta function
absorbs the uniform exponent pattern.

In [3]:
from q_kangaroo import etamake

em = etamake(pgf, 20)
print("Eta-quotient factors:", em["factors"])
print("q-shift:", em["q_shift"])

Eta-quotient factors: {1: -1}
q-shift: -1/24


The result $\{1: -1\}$ with $q$-shift $-1/24$ means:

$$\sum p(n)q^n = q^{-1/24} \cdot \eta(\tau)^{-1} = \frac{1}{(q;q)_\infty}$$

The $q^{-1/24}$ shift accounts for the difference between the eta
function $\eta(\tau) = q^{1/24}(q;q)_\infty$ and the bare infinite product.

## Jacprodmake: Jacobi Product Representation

The `jacprodmake` function searches for a Jacobi product representation
$\prod J(a_i, b_i)^{e_i}$ by looking for periodic residue-class patterns
in the `prodmake` exponents.

In [4]:
from q_kangaroo import theta4, jacprodmake

t4 = theta4(s, 30)
print("theta4 =", t4)
print()

jpm = jacprodmake(t4, 30)
print("Jacprodmake result:")
print("Factors:", jpm["factors"])
print("Scalar:", jpm["scalar"])
print("Exact:", jpm["is_exact"])

theta4 = 1 - 2*q + 2*q^4 - 2*q^9 + 2*q^16 - 2*q^25 + O(q^30)

Jacprodmake result:
Factors: {(1, 2): 1}
Scalar: 1
Exact: True


The result $J(1,2)^1$ confirms that $\theta_4(q) = J(1,2)$, the Jacobi
triple product with parameters $a=1$, $b=2$. The `is_exact` flag indicates
that the factorization accounts for all prodmake exponents.

## Mprodmake: Modified Product Representation

The `mprodmake` function extracts $(1+q^n)$ factors iteratively, using
the identity $(1+q^n) = (1-q^{2n})/(1-q^n)$. This is useful for series
that are products of $(1+q^n)$ terms.

In [5]:
from q_kangaroo import distinct_parts_gf, mprodmake

dpgf = distinct_parts_gf(s, 50)
print("Distinct parts gf:", distinct_parts_gf(s, 20))
print()

mp = mprodmake(dpgf, 20)
print("Mprodmake factors (m_n where f = prod (1+q^n)^{m_n}):")
print(mp)

Distinct parts gf: 1 + q + q^2 + 2*q^3 + 2*q^4 + 3*q^5 + 4*q^6 + 5*q^7 + 6*q^8 + 8*q^9 + 10*q^10 + 12*q^11 + 15*q^12 + 18*q^13 + 22*q^14 + 27*q^15 + 32*q^16 + 38*q^17 + 46*q^18 + 54*q^19 + O(q^20)

Mprodmake factors (m_n where f = prod (1+q^n)^{m_n}):
{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1}


Every $(1+q^n)$ exponent equals 1, confirming the classical formula:

$$\sum_{n=0}^{\infty} q(n)\, q^n = \prod_{k=1}^{\infty}(1 + q^k)$$

where $q(n)$ counts partitions of $n$ into distinct parts.

## Sift: Extracting Subsequences

The `sift(f, m, j)` function extracts the arithmetic subsequence
$\{a_{mn+j}\}$ from a series $f = \sum a_n q^n$. This is the key tool
for studying partition congruences and dissections.

In [6]:
from q_kangaroo import sift

pgf50 = partition_gf(s, 50)

# Extract p(5n+4): all coefficients divisible by 5
print("p(5n+4) coefficients:")
sift(pgf50, 5, 4)

p(5n+4) coefficients:


5 + 30*q + 135*q^2 + 490*q^3 + 1575*q^4 + 4565*q^5 + 12310*q^6 + 31185*q^7 + 75175*q^8 + 173525*q^9 + O(q^10)

In [7]:
# Extract p(7n+5): all coefficients divisible by 7
print("p(7n+5) coefficients:")
sift(pgf50, 7, 5)

p(7n+5) coefficients:


7 + 77*q + 490*q^2 + 2436*q^3 + 10143*q^4 + 37338*q^5 + 124754*q^6 + O(q^7)

Every coefficient in $p(5n+4)$ is divisible by 5, and every coefficient in
$p(7n+5)$ is divisible by 7 -- numerical evidence for Ramanujan's
congruences. The `sift` function is the building block behind the
automated `findcong` discovery tool.

## Qfactor: Polynomial Factoring

The `qfactor` function factors a finite q-polynomial into
$(1-q^i)$ factors. This inverts the multiplication of cyclotomic-like
factors.

In [8]:
from q_kangaroo import aqprod, qfactor

# (q;q)_3 = (1-q)(1-q^2)(1-q^3)
poly = aqprod(s, 1, 1, 1, 3, 10)
print("Polynomial:", poly)
print()

qf = qfactor(poly)
print("Qfactor result:")
print("Factors:", qf["factors"])
print("Scalar:", qf["scalar"])
print("Exact:", qf["is_exact"])

Polynomial: 1 - q - q^2 + q^4 + q^5 - q^6 + O(q^10)

Qfactor result:
Factors: {1: 1, 2: 1, 3: 1}
Scalar: 1
Exact: True


The factors $\{1: 1, 2: 1, 3: 1\}$ confirm that the polynomial is
$(1-q)^1 (1-q^2)^1 (1-q^3)^1$, recovering the original Pochhammer
factorization.

## Findlincombo: Linear Relations

The `findlincombo(target, candidates, topshift)` function searches for
a linear combination $\text{target} = \sum c_i \cdot g_i$ using exact
rational arithmetic (RREF over $\mathbb{Q}$).

**Euler's theorem** states that the number of partitions of $n$ into
distinct parts equals the number of partitions into odd parts. We can
verify this by showing that `odd_parts_gf = 1 * distinct_parts_gf`.

In [9]:
from q_kangaroo import findlincombo, odd_parts_gf

opgf = odd_parts_gf(s, 30)
dpgf30 = distinct_parts_gf(s, 30)

# Express odd_parts_gf as a linear combination of [distinct_parts_gf]
result = findlincombo(opgf, [dpgf30], 0)
print("Coefficients:", result)
print("Euler's theorem confirmed: odd_parts_gf = 1 * distinct_parts_gf")

Coefficients: [Fraction(1, 1)]
Euler's theorem confirmed: odd_parts_gf = 1 * distinct_parts_gf


The coefficient $[1]$ means `odd_parts_gf` $= 1 \cdot$ `distinct_parts_gf`,
confirming Euler's partition identity:

$$\prod_{k=0}^{\infty}\frac{1}{1-q^{2k+1}} = \prod_{k=1}^{\infty}(1+q^k)$$

## Findhom and Findpoly: Polynomial Relations

q-Kangaroo provides two tools for discovering polynomial relations:

- `findhom(series_list, degree, topshift)` -- finds **homogeneous**
  polynomial relations of exact degree $d$. The monomials are all products
  of exactly $d$ series from the list.

- `findpoly(x, y, deg_x, deg_y, topshift)` -- finds a bivariate polynomial
  $P(x, y) = \sum c_{ij} x^i y^j = 0$ of bounded degree.

We demonstrate `findpoly` by rediscovering the linear relation
`odd_parts_gf = distinct_parts_gf` as a polynomial identity:

In [26]:
from q_kangaroo import findhom

# findhom: find homogeneous polynomial relations of given degree
# Search for degree-1 (linear) relations among [odd_parts_gf, distinct_parts_gf]
relations = findhom([opgf, dpgf30], 1, 0)
print("Null space dimension:", len(relations))
print("Relation:", relations[0])
print("Meaning: -1*odd_parts + 1*distinct_parts = 0")

Null space dimension: 1
Relation: [Fraction(-1, 1), Fraction(1, 1)]
Meaning: -1*odd_parts + 1*distinct_parts = 0


In [10]:
from q_kangaroo import findpoly

# Search for P(x, y) = 0 where x = odd_parts_gf, y = distinct_parts_gf
x = odd_parts_gf(s, 50)
y = distinct_parts_gf(s, 50)

result = findpoly(x, y, 2, 2, 0)
print("Found relation:", result is not None)
if result:
    print("Degree in x:", result["deg_x"])
    print("Degree in y:", result["deg_y"])
    print("Coefficients:", result["coefficients"])

Found relation: True
Degree in x: 1
Degree in y: 1
Coefficients: [[Fraction(0, 1), Fraction(-1, 1)], [Fraction(1, 1), Fraction(0, 1)]]


The coefficient grid $c_{ij}$ with $c_{01} = -1$ and $c_{10} = 1$ means:

$$P(x, y) = x - y = 0$$

i.e., the two generating functions are identical -- Euler's theorem expressed
as a polynomial identity. Although we searched up to degree 2, `findpoly`
returns the simplest (lowest-degree) relation.

For more complex examples, `findpoly` discovers nontrivial algebraic curves
relating modular forms and eta-quotients -- a key tool in the theory
of modular equations.

## Findcong: Automated Congruence Discovery

The `findcong` function systematically searches for congruences
$a(mn + r) \equiv 0 \pmod{d}$ by sifting and testing divisibility
at each residue class.

In [11]:
from q_kangaroo import findcong

# Search across a wide range of primes
pgf100 = partition_gf(s, 100)
congs = findcong(pgf100, [5, 7, 11, 13, 17, 19, 23])

print("Congruences found:")
for c in congs:
    print(f"  p({c['modulus']}n + {c['residue']}) == 0 (mod {c['divisor']})")

print()
print("No congruences for moduli 13, 17, 19, 23 -- Ramanujan's three are special!")

Congruences found:
  p(5n + 4) == 0 (mod 5)
  p(7n + 5) == 0 (mod 7)
  p(11n + 6) == 0 (mod 11)

No congruences for moduli 13, 17, 19, 23 -- Ramanujan's three are special!


Only the three Ramanujan primes 5, 7, and 11 yield congruences for the
partition function among the primes tested. This is not coincidence:
Ramanujan-type congruences $p(\ell n + \delta_\ell) \equiv 0 \pmod{\ell}$
exist only for $\ell \in \{5, 7, 11\}$ among primes.

## Summary

This notebook demonstrated the complete series analysis pipeline:

| Tool | Purpose |
|------|:--------|
| `prodmake` | Andrews' series-to-product algorithm |
| `etamake` | Dedekind eta-quotient representation |
| `jacprodmake` | Jacobi product representation |
| `mprodmake` | Modified $(1+q^n)$ product decomposition |
| `sift` | Extract arithmetic subsequences $a(mn+j)$ |
| `qfactor` | Factor polynomials into $(1-q^i)$ factors |
| `findlincombo` | Linear combination search over $\mathbb{Q}$ |
| `findhom` | Homogeneous polynomial relations |
| `findpoly` | Bivariate polynomial relation $P(x,y)=0$ |
| `findcong` | Automated congruence discovery |

**Typical research workflow:**

1. Compute a series to high precision
2. Use `prodmake` / `etamake` to identify its product structure
3. Use `sift` to investigate arithmetic subsequences
4. Use `findcong` to discover congruences automatically
5. Use `findlincombo` / `findhom` / `findpoly` to discover relations
   between multiple series

For related topics, see:
- **Getting Started** (`getting_started.ipynb`): Core API introduction
- **Partition Congruences** (`partition_congruences.ipynb`): Rank and crank analysis
- **Theta Identities** (`theta_identities.ipynb`): Jacobi theta functions